In [114]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import preprocessing # use to convert text into value

In [115]:
dataset =pd.read_csv("train.csv")


In [116]:
dataset.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [117]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [118]:
# in age we are having missing value so we find the avg value 

In [119]:
dataset["Age"].mean()

32.69985376827896

In [120]:
new_age_var=np.where(dataset["Age"].isnull(),32,dataset["Age"])

In [121]:
dataset["Age"]=new_age_var

In [122]:
label_encoder=preprocessing.LabelEncoder()

In [123]:
encoder_sex=label_encoder.fit_transform(dataset["Sex"])#transform converts text inti numeric

In [124]:
tree_model=tree.DecisionTreeClassifier()

In [136]:
tree_model.fit(X=pd.DataFrame(encoder_sex),y=dataset["Survived"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [137]:
# we are passing the independent variable sex is IDV and Y is dependent variable that is survive
# we are created model between the sex and survived

In [138]:
with open("Dtree1.dot",'w') as f: # dtree means name of decision tree it always store in dot format w means write what write that is f
   f=tree.export_graphviz(tree_model,feature_names=["Sex"],out_file=f);
# tree is function and grapviz is a interface of tree model,where feature naem is inpendent varialbe,out_file means where we store
    

In [139]:
Predictor=pd.DataFrame([encoder_sex,dataset["Pclass"],dataset["Age"],dataset["Fare"]]).T

In [140]:
tree_model=tree.DecisionTreeClassifier(max_depth=8)


In [141]:
tree_model.fit(X=Predictor,y=dataset["Survived"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [142]:
with open("Dtree_1_1.dot",'w') as f:
    f=tree.export_graphviz(tree_model,feature_names=["Sex","Pclass","Age","Fare"],out_file=f);

In [143]:
dataset1=pd.read_csv("test.csv")

In [144]:
# avg age to replce missing values

In [146]:
new_age_var=np.where(dataset1["Age"].isnull,28,dataset1["Age"])

In [147]:
dataset1["Age"]=new_age_var

In [148]:
encoder_sex_test=label_encoder.fit_transform(dataset1["Sex"])#transform converts text inti numeric

In [149]:
test_feature=pd.DataFrame([encoder_sex_test,dataset1["Pclass"],dataset1["Age"],dataset1["Fare"]]).T

In [150]:
test_preds=tree_model.predict(X=test_feature) 


In [151]:
Predict_output=pd.DataFrame({"PassengerId":dataset1["PassengerId"],"Survived":test_preds})
# using PassengerId and survived we predict the test output

In [152]:
Predict_output.to_csv("Output.csv",index=False);

In [153]:
# it is essembling of Desion tree to find the important variable Random Forest algo

In [154]:
from sklearn.ensemble import RandomForestClassifier

In [156]:
dataset.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [162]:
label_encoder=preprocessing.LabelEncoder()

In [163]:
dataset["Sex"]=label_encoder.fit_transform(dataset["Sex"])

In [164]:
dataset["Embarked"]=label_encoder.fit_transform(dataset["Embarked"])

In [165]:
rf_model=RandomForestClassifier(n_estimators=100,max_features=2,oob_score=True)


In [166]:
features=["Sex","Pclass","SibSp","Embarked","Age","Fare"]#out of this IDV we need to find the important variable

In [167]:
rf_model.fit(X=dataset[features],y=dataset["Survived"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [168]:
#using these we find accuraccy code

In [169]:
print("OOB Accuracy:")
print(rf_model.oob_score_);

OOB Accuracy:
0.8098987626546682


In [170]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)

Sex 0.27836174979639244
Pclass 0.08708403698227926
SibSp 0.05042488645870971
Embarked 0.03276715704948112
Age 0.2669216886263477
Fare 0.28444048108678976


In [171]:
# lets draw a decision tree for the variable which have highest value like Sex,Age.Fare

In [172]:
from sklearn import tree

In [173]:
tree_model=tree.DecisionTreeClassifier()

In [174]:
tree_model=tree.DecisionTreeClassifier(max_depth=6,max_leaf_nodes=10)

In [175]:
predictors=pd.DataFrame([dataset["Sex"],dataset["Age"],dataset["Fare"]]).T

In [176]:
tree_model.fit(X=predictors,y=dataset["Survived"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [177]:
with open("Dtree_1.dot",'w') as f:
    f=tree.export_graphviz(tree_model,feature_names=["Sex","Age","Fare"],out_file=f);